# Прикладное программное обеспечение
#### Python для извлечения и обработки данных


## API - telegram, vk, бонусы
Социлогия | 1 курс | 3 модуль | Семинар 7

*Авторы: Валентин Бирюков, НИУ ВШЭ*

Это занятие является скорее показательным навыком того, что вы уже можете делать, используя полученные знания, а так же cntl+c-cntl+v с гугла. В ней проведены каркасы и пояснения к некоторым базовым оболочкам кода для взаимодействия с Вк и телеграмм.

## Ссылки

Возпользовавшись источниками ниже данный блокнот можно и вообще пропустить =)
Как и многое другое по программированию все уже возможно где то сделано, нам же предстоит заиспользовать это для своих нужд. Список ниже - ссылки и статьи на основе которых написан данный блокнот.

1) https://python-telegram-bot.org/ - лендинг почти официальной библиотеки модуля для телеграм

2) https://github.com/python-telegram-bot/python-telegram-bot - он же на гитхабе

3) https://habr.com/ru/post/448310/ - туториал по нему, в придачу еще искусственный интеллект.

4) https://github.com/eternnoir/pyTelegramBotAPI - похожая версия

5) https://habr.com/ru/post/442800/ - другой туториал. по похожему модулю, только 


## Быстрый старт

Самая простая и описанная часть. Очень коротко: нужно найти бота @BotFather, написать ему /start, или /newbot, заполнить поля, которые он спросит (название бота и его короткое имя), и получить сообщение с токеном бота и ссылкой на документацию. Токен нужно сохранить, желательно надёжно, так как это единственный ключ для авторизации бота и взаимодействия с ним.

Так же нужно поставить первую библиотеку.

In [4]:
!pip install python-telegram-bot

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# настройки для бота:
API_TOKEN = 'Token'
BOT_NAME = 'HseSocExperimentBot'

Напишем простого бота который запоминает имя, а потом отпрашивется немного поучиться.

In [25]:
import logging # модуль для логирования, будет писать нам в консоль о происходящем
from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove) # методы для работы ответа "кнопочками"
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler
)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO) 

logger = logging.getLogger(__name__)

In [26]:
GROUP, NAKOP = range(2) # состояния в которых пребывает бот
# GROUP - мы спроисли и получили номер группы
# NAKOP - докапываемся на счет накопа

In [27]:
def start(update, context):
    group_number = [list(map(str, range(191, 199)))] # Варианты доступные для ответа: числа 191-198
    hellow_text = ''' 
Привет! я бот - вышкинец. Я пока только учусь.
Отправь мне /cancel чтобы отправить меня ботать.
    
Расскажи из какой ты группы?
    ''' 
    update.message.reply_text( # отправим сообщение и предложим на выбор варианты из списка
        hellow_text,
        reply_markup=ReplyKeyboardMarkup(group_number, one_time_keyboard=True)
    )
    return GROUP # мы переходим в следующее сосотяние - в нем заберем номер группы и спросим имя

In [28]:
def group(update, context):
    user = update.message.from_user
    group_id = update.message.text
    # Выведем номер группы пользователя себе локально
    logger.info("Получен номер группы пользователя  %s: %s", user.first_name, group_id)
    update.message.reply_text(
        'ох, слышал в группе {} нелегко. Вот какой у тебя накоп?'.format(group_id), 
        reply_markup=ReplyKeyboardRemove() # уберем клавиатуру с прошлого раза, теперь надо написать текстом.
    )

    return NAKOP

In [37]:
import time

def nakop(update, context):
    # 
    user = update.message.from_user
    mark_text = update.message.text
    logger.info("У пользователя %s накоп: %s", user.first_name, mark_text)
    # теперь реализуем такую логику:
    # если пользователь отправил и не особо то число - повторим процедуру еще раз
    # если все таки число - то посмотрим хватает ли на зачет =)
    try: # эта конструкция спасет нас от ошибок.
        mark = float(mark_text) # если это не число и преобразованиие не получится
    except: # то мы провалимся в этот блок
        update.message.reply_text(
            'Это разве накоп? да это и не число то особо. Давай еще раз: какой у тебя накоп?'
        )
        return NAKOP
    
    # если дошли сюда - значит в mark лежит число
    if mark > 3.5:
        update.message.reply_text('Ого! да у тебя уже зачет. А у меня нет. пойду ботать.')
        return ConversationHandler.END
    else:
        update.message.reply_text('Ой, какой то он маленький. Давай я подожду чуть чуть.')
        time.sleep(10)
        update.message.reply_text('А теперь какой?')
        return NAKOP

In [38]:
'''Это системные функции для обработки ошибок и исключений, а также конца разговора. на всякий случай'''

def cancel(update, context):
    user = update.message.from_user
    logger.info("Пользователь %s закончил разговор", user.first_name)
    update.message.reply_text('Ну ладно, пойду ботать',reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

In [39]:
def main():
    # создаем канал для обещния с телеграмом. потребуется ключ для общения с ботом
    updater = Updater(API_TOKEN, use_context=True)
    dp = updater.dispatcher

    # конфигурируем какие у нас могут быть состояния
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)], # начинамем с того что описали в функции start
        states={
            GROUP: [MessageHandler(Filters.regex('^19.$'), group)],
            # описываем состояние "группы" - по сути то что описано в функции group
            # фильтруем в этом состоянии все сообщения которые не начинаются на 19*
            NAKOP: [MessageHandler(Filters.text, nakop)]
            # наше второе состояние - докапываем до накопленной оценки
        },
        fallbacks=[CommandHandler('cancel', cancel)]
        # что делать в случае отмены - завершаем диалог функцией cancel
    )

    # собираем и запускаем нашу конфигурацию
    dp.add_handler(conv_handler)
    # log all errors
    dp.add_error_handler(error)
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [40]:
# эта ячейка зависнет при запуске - потому что пока она запущена
# она контролирует работу бота в телеграме. если она не запущена - бот в телеграме не отвечает
main()

2020-03-15 21:55:15,500 - __main__ - INFO - Получен номер группы пользователя  Biryukov: 196
2020-03-15 21:55:18,536 - __main__ - INFO - У пользователя Biryukov накоп: 1
2020-03-15 21:55:34,094 - __main__ - INFO - У пользователя Biryukov накоп: 2
2020-03-15 21:55:47,773 - __main__ - INFO - У пользователя Biryukov накоп: 4
2020-03-15 21:57:39,888 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


## TgBot а теперь с ИИ

Концептуально нового  здесь ничего нет. по сути демонстрация и пересказ одного из туториалов выше

In [44]:
!pip install apiai

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [47]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import apiai, json
updater = Updater(token=API_TOKEN) # Токен API к Telegram
dispatcher = updater.dispatcher

/Users/biryuk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
#Суть то же самое приветствие
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

In [49]:
API_TOKEN_FOR_AI = 'TOKEN'
client = apiai.ApiAI(API_TOKEN_FOR_AI)


По сути в нашем боте не будет никакой логики. Он будет принимать текст, оттправлять его в DialogFlow, смотреть что ответит бот там, а потом отправлять данный той машинерией ответ назад пользователю. Зато красиво.
Инструкция как получить ключ: https://habr.com/ru/post/346606/

In [50]:
def textMessage(bot, update):
    request = client.text_request() # Токен API к Dialogflow
    request.lang = 'ru' # На каком языке будет послан запрос
    request.session_id = 'HseBot' # ID Сессии диалога (нужно, чтобы потом учить бота)
    request.query = update.message.text # Посылаем запрос к ИИ с сообщением от юзера
    responseJson = json.loads(request.getresponse().read().decode('utf-8'))
    response = responseJson['result']['fulfillment']['speech'] # Разбираем JSON и вытаскиваем ответ
    # Если есть ответ от бота - присылаем юзеру, если нет - бот его не понял
    if response:
        bot.send_message(chat_id=update.message.chat_id, text=response)
    else:
        bot.send_message(chat_id=update.message.chat_id, text='Я Вас не совсем понял!')

In [51]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

2020-03-15 22:24:19,258 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


## VK API

Возможно в этом месте вы задумаетесь как же находить удобные модули. Вот например для вк. Или для твиттера. Или для инстаграмма. Спасает https://github.com/ или Google. Но первый обычно удобнее. В поиске пишем более мнее внятно и коротко что мы хотим, потом добавляем язык на котором нас это интересует (в нашем случае это python или jupiter notebook) и вуаля. Чем больше звезд - тем более популярна и признана библиотека. Использовать такие проще и потятнее, а еще у них чаще бывает внятная документация.

Популярна вот эта:
https://github.com/python273/vk_api

Хотя не менее популярна вот эта: https://github.com/voronind/vk и ее мы тоже посмотрим.

Hазницы по факту почти никакой. Для личного быстрого пользования проще авторизоваться и использовать первую. 

In [54]:
! pip install vk_api vk

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Самое сложное в начале работы с вк - получить несчастный ключ доступа. Оно и хорошо с другой стороны - этот ключ доступа дает полное право на работу от имени любого человека, все равно что знание логина и пароля. Мы обойдемся без этого. Два наиболее простых варианта - взять включ от приложения (и для этого надо самому завести приложение). Или ключ от сообщества. Оба берутся в настройках и просто копируются.

ВНИМАНИЕ: ключ доступа - все равно что логин и пароль. не передедвайте их никому

In [109]:
import vk # загружаем модуль, второй
API_TOKEN_VK = 'TOKEN'
session = vk.Session(access_token=API_TOKEN_VK)
my_api = vk.API(session, v='5.103')


Теперь у нас есть возможность использовать информацию с вк по средствам my_api - своего рода "тунеля" до вконтакте. Все интересующие функции можно найти в https://vk.com/dev/methods

Напомню. мы используем ключ доступа приложения, а потому толком не авторизовались. Мы не можем писать сообщения, публикации на стене, не можем смотреть закрытые профили, но можем смотреть на пользователей где это возможно =)

Терепрь у нас есть метод модуль который позволяет взаимодействовать с вк! погнали.
В целях эксперимента посмотрим какую нибудь информацию о пользователе. Например, список другей первого круга, второго и третьго.

In [120]:
user_to_prepare = 'valentin.biryukov' # пользователь, с которого начнем. 

user_info = my_api.users.get(user_ids=user_to_prepare) # посмотрим что видно про пользователя
print(user_info)
start_id = user_info[0]['id']

[{'id': 21785564, 'first_name': 'Valentin', 'last_name': 'Biryukov', 'is_closed': False, 'can_access_closed': True}]


In [122]:
friends_first_hand = my_api.friends.get(user_id=start_id) # тот самый идентификатор
print(friends_first_hand)
friends_first_hand_id = friends_first_hand['items']

{'count': 83, 'items': [398045, 409037, 1538693, 2956433, 4228387, 4516987, 5317749, 5334461, 5412679, 10348887, 10873145, 11164850, 13275626, 14204056, 15945852, 17336364, 20580174, 22304277, 24874560, 24976336, 30606127, 30816900, 34248421, 34746243, 36623405, 38459100, 38830624, 38900678, 44898863, 47402039, 47672038, 49915195, 50489710, 51121895, 52203550, 53847565, 54782818, 56726638, 59536344, 61657552, 62902082, 66865352, 68603392, 68812424, 71542865, 72937909, 76702064, 85427885, 88293148, 95437665, 97341087, 106480301, 107098336, 107916896, 107920824, 112162069, 114776804, 114904924, 122307165, 124821194, 125824662, 145762347, 148396546, 152426005, 156561760, 161176665, 165968752, 177372466, 209463582, 209662408, 213734038, 227227659, 262217495, 271656949, 275633587, 283975933, 295142987, 317187625, 321409001, 370549931, 560118043, 566953060, 574103916]}


In [126]:
# Давайте соберем пользователей второго круга!
friends_second_hand_id = []
private_accaunts = []
for item in friends_first_hand_id:
    try:
        ids = my_api.friends.get(user_id=item)['items']
        friends_second_hand_id.extend(ids)
    except:
        print('Пользователь с id={}, имеет приватный аккаунт!'.format(item))
        private_accaunts.append(item)
    
friends_second_hand_id = list(set(friends_second_hand_id))
print('На втором кругу уже {} друзей'.format(len(friends_second_hand_id)))

# на этом можно остановиться если их много

Пользователь с id=24976336, имеет приватный аккаунт!
Пользователь с id=30606127, имеет приватный аккаунт!
Пользователь с id=34248421, имеет приватный аккаунт!
Пользователь с id=44898863, имеет приватный аккаунт!
Пользователь с id=49915195, имеет приватный аккаунт!
Пользователь с id=56726638, имеет приватный аккаунт!
Пользователь с id=62902082, имеет приватный аккаунт!
Пользователь с id=68812424, имеет приватный аккаунт!
Пользователь с id=107920824, имеет приватный аккаунт!
Пользователь с id=125824662, имеет приватный аккаунт!
Пользователь с id=165968752, имеет приватный аккаунт!
Пользователь с id=227227659, имеет приватный аккаунт!
Пользователь с id=295142987, имеет приватный аккаунт!
Пользователь с id=317187625, имеет приватный аккаунт!
На втором кругу уже 32218 друзей


Давайте ограничимся первым кругом и посчитаем какие нибудь статистики по друзьям.

In [130]:
freinds_to_anallize = [ item for item in friends_first_hand_id if item not in private_accaunts]
#оставили только открытые аккаунты

infos = my_api.users.get(
    user_ids=freinds_to_anallize,
    fields=['sex', 'city', 'country', 'contacts', 'universities', 'relation', 'career']
)
print(infos[:3])
# к сожалению большая часть пунктов будет скрыта при поиске, но нас это не очень остановит

[{'id': 398045, 'first_name': 'Yulia', 'last_name': 'Volkova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'id': 1, 'title': 'Moscow'}, 'country': {'id': 1, 'title': 'Russia'}}, {'id': 409037, 'first_name': 'Olga', 'last_name': 'Podolskaya', 'is_closed': False, 'can_access_closed': True, 'sex': 1}, {'id': 1538693, 'first_name': 'Gleb', 'last_name': 'Evstropov', 'is_closed': False, 'can_access_closed': True, 'sex': 2, 'city': {'id': 1, 'title': 'Moscow'}, 'country': {'id': 1, 'title': 'Russia'}}]


In [133]:
import collections

# посчитаем соотношение полов:
sex = collections.Counter(item.get('sex') for item in infos)
print("Сколько людей какого пола в друзьях (2 - мужской, 1 - женский):", sex)

# и по странам: 
country = collections.Counter(item.get('country', {}).get('title') for item in infos)
print("Статистика по странам:", country)

# и по городам: 
city = collections.Counter(item.get('city', {}).get('title') for item in infos)
print("Статистика по странам:", city)

Сколько людей какого пола в друзьях (2 - мужской, 1 - женский): Counter({2: 37, 1: 32})
Статистика по странам: Counter({'Russia': 59, None: 9, 'Germany': 1})
Статистика по странам: Counter({'Moscow': 48, None: 10, 'Saint Petersburg': 4, 'Orenburg': 3, 'Berlin': 1, 'Gus-Khrustalny': 1, 'Tver': 1, 'Odintsovo': 1})


In [135]:
# Соберем последние записи со стен друзей
posts = []
for item in freinds_to_anallize:
    try:
        posts.append(my_api.wall.get(
            owner_id=item,
            count=10, #ограничимся последним 10 записями
            filter='owner', # берем только записи от самого пользователя
            extended=1,
        ))
    except:
        print('User {} hide his wall'.format(item))
print(posts[:2])

User 398045 hide his wall
User 409037 hide his wall
User 1538693 hide his wall
User 5317749 hide his wall
User 13275626 hide his wall
User 15945852 hide his wall
User 17336364 hide his wall
User 24874560 hide his wall
User 38459100 hide his wall
User 47402039 hide his wall
User 47672038 hide his wall
User 50489710 hide his wall
User 52203550 hide his wall
User 54782818 hide his wall
User 71542865 hide his wall
User 76702064 hide his wall
User 88293148 hide his wall
User 95437665 hide his wall
User 107098336 hide his wall
User 107916896 hide his wall
User 112162069 hide his wall
User 114776804 hide his wall
User 122307165 hide his wall
User 124821194 hide his wall
User 148396546 hide his wall
User 177372466 hide his wall
User 209463582 hide his wall
User 209662408 hide his wall
User 213734038 hide his wall
User 321409001 hide his wall
User 574103916 hide his wall
[{'count': 1457, 'items': [{'id': 4325, 'from_id': 2956433, 'owner_id': 2956433, 'date': 1583792624, 'post_type': 'post', 'te

Вместо послесловия:

Работа с личным ключем может дать заметно больше информации и простора для аналитики. большая часть профилей логично скрыта от такого обхода.

В рамках самостоятельного исследования можете обратить внимание еще на функционал групп, который позволяет аналогично так же строить диалоговые системы от имени группы.

А так же можете обратить внимания на методы связанные с получаением лайков, коментов и репостов, там тоже много полезного.